# Libraries

In [1]:
import  pandas as pd
import  numpy as np
import  plotly.express as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go

from    sklearn.model_selection import train_test_split
from    sklearn.ensemble import RandomForestClassifier
from    sklearn.preprocessing import StandardScaler
from    sklearn.metrics import accuracy_score
from    sklearn.metrics import confusion_matrix
from    sklearn.metrics import precision_score
from    sklearn.metrics import recall_score
from    sklearn.metrics import f1_score
from    sklearn.metrics import ConfusionMatrixDisplay
from    sklearn.metrics import RocCurveDisplay
from    sklearn.metrics import roc_auc_score
from    sklearn.model_selection import cross_val_score
from    sklearn.impute import SimpleImputer
from    imblearn.over_sampling import SMOTE
from    sklearn.model_selection import RandomizedSearchCV
from    sklearn.model_selection import cross_validate
from    sklearn.model_selection import learning_curve

In [2]:
RANDOM_SEED = 0

In [3]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction


c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [76]:
params = {
    "model_parameters": {
        "n_estimators": [15, 20, 30, 50, 100, 200],
        "criterion": ["gini", "entropy"],
        "max_depth": [3, 6, 9],
        "min_samples_split": [0.2, 0.3, 0.5],
        "min_samples_leaf": [2]
    },
    "cross_validation": {
        "n_iterations": 3,
        "folders": 3,
        "scores": ["accuracy", "recall"]
    },
    "target": "class",
    "train_size": 0.8
}


# Train and validation

## Load data

In [77]:
df_train_original = pd.read_csv('data\\original_data\\air_system_previous_years.csv',
                                encoding='utf-8',
                                sep=',',
                                na_values=['na']
                                )
df_train_original.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\original_data\\air_system_previous_years.csv'

In [6]:
df_train_original['class'].value_counts()

class
neg    59000
pos     1000
Name: count, dtype: int64

In [7]:
df_test_original = pd.read_csv('data\\original_data\\air_system_present_year.csv',
                               encoding='utf-8',
                               sep=',',
                               na_values=['na']
                               )
df_test_original.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,60,0.0,20.0,12.0,0.0,0.0,0.0,0.0,0.0,...,1098.0,138.0,412.0,654.0,78.0,88.0,0.0,0.0,0.0,0.0
1,neg,82,0.0,68.0,40.0,0.0,0.0,0.0,0.0,0.0,...,1068.0,276.0,1620.0,116.0,86.0,462.0,0.0,0.0,0.0,0.0
2,neg,66002,2.0,212.0,112.0,0.0,0.0,0.0,0.0,0.0,...,495076.0,380368.0,440134.0,269556.0,1315022.0,153680.0,516.0,0.0,0.0,0.0
3,neg,59816,NaN,1010.0,936.0,0.0,0.0,0.0,0.0,0.0,...,540820.0,243270.0,483302.0,485332.0,431376.0,210074.0,281662.0,3232.0,0.0,0.0
4,neg,1814,NaN,156.0,140.0,0.0,0.0,0.0,0.0,0.0,...,7646.0,4144.0,18466.0,49782.0,3176.0,482.0,76.0,0.0,0.0,0.0


In [8]:
df_test_original['class'].value_counts()

class
neg    15625
pos      375
Name: count, dtype: int64

# Preprocessing

In [9]:
target   = params['target']
features = df_train_original.drop(columns=target).columns

# Adjust features columns type
df_train_original[features] = df_train_original[features].astype(float)
df_test_original[features]  = df_test_original[features].astype(float)

# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_train_original[target] = df_train_original[target].map(map_class)
df_test_original[target]  = df_test_original[target].map(map_class)

In [10]:
# Use the median to replace NaN values
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(df_train_original[features])

# Fill train NaN with median imputer
df_train_original[features] = imputer.transform(df_train_original[features])

# Fill test NaN with same train imputer
df_test_original[features] = imputer.transform(df_test_original[features])

In [11]:
# Balance train data minority class
X_train = df_train_original[features]
y_train = df_train_original[target]

oversample = SMOTE(sampling_strategy='minority', random_state=RANDOM_SEED)
X_train_balanced, y_balanced = oversample.fit_resample(X_train, y_train)

# Train dataframe with balanced data
df_train_balanced = X_train_balanced.join(y_balanced)

In [12]:
df_train_balanced['class'].value_counts()

class
0    59000
1    59000
Name: count, dtype: int64

In [13]:
# Remove zero variance train columns
columns_to_drop = []
X_train = df_train_balanced.drop(columns=target)
for i in X_train:
    if X_train[i].std() == 0:
        columns_to_drop.append(i)
dataframe_train_zero_std = df_train_balanced.drop(columns=columns_to_drop).copy()

In [14]:
# Remove the same column on test
dataframe_test_zero_std = df_test_original.drop(columns=columns_to_drop).copy()

In [15]:
features_zero_std = dataframe_train_zero_std.drop(columns=target).columns

In [16]:
# Scale data
scaler = StandardScaler()
scaler.fit(dataframe_train_zero_std[features_zero_std])

# Data scaled
X_train_scaled = scaler.transform(dataframe_train_zero_std[features_zero_std])
X_test_scaled  = scaler.transform(dataframe_test_zero_std[features_zero_std])

df_train_scaled = pd.DataFrame(X_train_scaled, columns=features_zero_std, index=dataframe_train_zero_std.index)
df_train_scaled = dataframe_train_zero_std[[target]].join(df_train_scaled)

In [17]:
df_train_scaled.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,-0.639811,-0.144124,3.232075,-0.002923,-0.060063,-0.069258,-0.055244,-0.150456,-0.300809,...,-0.312351,-0.337995,-0.424548,-0.479737,-0.479961,-0.420793,-0.318991,-0.216459,-0.036498,-0.047793
1,0,-0.743136,-0.144124,-0.309400,-0.002929,-0.060063,-0.069258,-0.055244,-0.150456,-0.300809,...,-0.543593,-0.520317,-0.542902,-0.536831,-0.539800,-0.447587,-0.302673,-0.205583,-0.036498,-0.047793
2,0,-0.724237,-0.144124,-0.309400,-0.002930,-0.060063,-0.069258,-0.055244,-0.150456,-0.300809,...,-0.584251,-0.530870,-0.506741,-0.495062,-0.485322,-0.420770,-0.304313,-0.212733,-0.036498,-0.047793
3,0,-0.821378,-0.144124,-0.309400,-0.002931,-0.060063,-0.036437,-0.055244,-0.150456,-0.300809,...,-0.662489,-0.623249,-0.624042,-0.599269,-0.578716,-0.475890,-0.368058,-0.216459,0.620282,2.826032
4,0,-0.677277,-0.144124,-0.309398,-0.002916,-0.060063,-0.069258,-0.055244,-0.150456,-0.300809,...,-0.486959,-0.490408,-0.511914,-0.510935,-0.495162,-0.367214,-0.077267,-0.207628,-0.036498,-0.047793


# Train

In [18]:
# Create and fit model
model = RandomForestClassifier(random_state=RANDOM_SEED, verbose=0)
model.fit(df_train_scaled[features_zero_std], df_train_scaled[target])

RandomForestClassifier(random_state=0)

## Tuning

In [19]:
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=params['model_parameters'],
                                   verbose=3,
                                   cv=params['cross_validation']['folders'],
                                   n_iter=params['cross_validation']['n_iterations'],
                                   random_state=RANDOM_SEED,
                                   scoring='accuracy'
                                   )
random_search.fit(df_train_scaled[features_zero_std], df_train_scaled[target])
# Define best parameters and apply on a new model
best_params = random_search.best_params_
print(f'\nBest parameters:\n{best_params}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END criterion=entropy, max_depth=6, min_samples_leaf=2, min_samples_split=0.5, n_estimators=15;, score=0.937 total time=   1.1s
[CV 2/3] END criterion=entropy, max_depth=6, min_samples_leaf=2, min_samples_split=0.5, n_estimators=15;, score=0.937 total time=   1.1s
[CV 3/3] END criterion=entropy, max_depth=6, min_samples_leaf=2, min_samples_split=0.5, n_estimators=15;, score=0.936 total time=   1.1s
[CV 1/3] END criterion=gini, max_depth=3, min_samples_leaf=2, min_samples_split=0.3, n_estimators=100;, score=0.946 total time=  10.3s
[CV 2/3] END criterion=gini, max_depth=3, min_samples_leaf=2, min_samples_split=0.3, n_estimators=100;, score=0.948 total time=  10.3s
[CV 3/3] END criterion=gini, max_depth=3, min_samples_leaf=2, min_samples_split=0.3, n_estimators=100;, score=0.947 total time=  10.4s
[CV 1/3] END criterion=entropy, max_depth=6, min_samples_leaf=2, min_samples_split=0.2, n_estimators=50;, score=0.947 total t

In [20]:
model_best_params = RandomForestClassifier(**best_params, random_state=RANDOM_SEED)
model_best_params.fit(df_train_scaled[features_zero_std], df_train_scaled[target])

RandomForestClassifier(criterion='entropy', max_depth=6, min_samples_leaf=2,
                       min_samples_split=0.2, n_estimators=50, random_state=0)

## Cross-validation

In [22]:
cv_results = cross_validate(estimator=model_best_params,
                            X=df_train_scaled[features_zero_std],
                            y=df_train_scaled[target],
                            scoring=params['cross_validation']['scores'],
                            cv=params['cross_validation']['folders'],
                            verbose=3,
                            return_train_score=True,
                            error_score=np.nan
                            )

[CV] END  accuracy: (train=0.948, test=0.947) recall: (train=0.965, test=0.964) total time=  11.9s
[CV] END  accuracy: (train=0.947, test=0.948) recall: (train=0.964, test=0.964) total time=  12.4s
[CV] END  accuracy: (train=0.947, test=0.946) recall: (train=0.963, test=0.961) total time=  13.4s


In [23]:
metrics = params['cross_validation']['scores']
metrics

['accuracy', 'recall']

In [24]:
cv_results['train_accuracy'].mean()

np.float64(0.947652545333525)

In [25]:
cv_results['test_accuracy'].mean()

np.float64(0.9471694902251144)

In [26]:
for score in metrics:
    train_mean = cv_results[f'train_{score}'].mean()
    test_mean = cv_results[f'test_{score}'].mean()
    print(train_mean, test_mean)

0.947652545333525 0.9471694902251144
0.9639661018333752 0.9632881519470097


# Predict

In [78]:
df_test_scaled = pd.DataFrame(X_test_scaled, columns=features_zero_std, index=dataframe_test_zero_std.index)
df_test_scaled = dataframe_test_zero_std[[target]].join(df_test_scaled)

X_test = df_test_scaled[features_zero_std]
y_test = df_test_scaled[target]

In [79]:
y_prob_predict  = model.predict_proba(X_test)[:, 1]
df_prob_predict = pd.DataFrame(y_prob_predict, columns=['y_prob_predict'])
df_prob_predict = df_prob_predict.join(y_test)
df_prob_predict.rename(columns={'class': 'y_test'}, inplace=True)
df_prob_predict = df_prob_predict.join(df_test_scaled[features_zero_std])

In [80]:
cutoff = 0.3
df_prob_predict['y_predict'] = np.where(df_prob_predict['y_prob_predict'] >= cutoff, 1, 0)

In [82]:
df_prob_predict['y_predict'].value_counts()

y_predict
0    15461
1      539
Name: count, dtype: int64